# LLAMA3 Fine-Tuning using LoRA

In [1]:
import huggingface_hub

ACCESS_TOKEN = ""
huggingface_hub.login(ACCESS_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Gamzadole\.cache\huggingface\token
Login successful


In [2]:
# 모델과 데이터셋

base_model = "beomi/Llama-3-Open-Ko-8B"

kr_dataset = "Bingsu/ko_alpaca_data"

In [3]:
# 필요 라이브러리
import os
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer

In [ ]:
# "beomi/Llama-3-Open-Ko-8B" 의 model과 tokenizer 가져오기

model = AutoModelForCausalLM.from_pretrained(base_model, load_in_4bit=True, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
prompt = "건강을 유지하기 위한 세 가지 팁을 알려주세요."

encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
model_inputs = encoded_input.to('cuda')

generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)

decoded_output = tokenizer.batch_decode(generated_ids)

print(decoded_output[0].replace(prompt, ""))

c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\bitsandbytes\nn\modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<|begin_of_text|> 첫 번째는요?<|end_of_text|><|begin_of_text|><|begin_of_text|>아울러 신차를 구매할 때 무이자 할부와 보증금 없이 저스탑 레이스를 운영하는 등 다양한 할부방식을 실시하고 있다. 또 보험처리 가능하고 신용카드 결제도 가능해 구매 고객들의 선택의 폭이 넓혀졌다.<|end_of_text|> 이와 관련, 전일 5개 신문들은 '정상회담 대화록 실종 사건의 전말'이란 제목을 뽑아 다음과 같은 추궁 기사들을 보도했다.
 ①(동아일보) 정상회담 대화록 대통령기록관 무단반출 누군일까…원본 폐기 논란 재점화.
  ②(조선일보) 대통령기록관이 대화록을 왜 몰래 빼돌렸나.
  ③(한경·서울) 정상회담 대화록이 대통령기록관에 있었나?.
  ④(한겨레) ‘임기 중 기록물은 대통령 재임 시 무기한 보존’ 국정원 입.
  ⑤(한국) ‘대화록 없었다’ 국정원 해명 “대화록 폐기” vs 대통령기록관 “이전 없었다”(본지 보도).
 이들 5개 신문은 원론적으로는 대화록 원본의 존재 여부와 대화록이 대통령기록관으로 이관된 사실이 없다는 국정원의 해명이 맞는지 의문을 제기한 보도들이었다.
 물론 당시(2012년 3월) 열린 정상회담 개최 직전 청와대 이지원시스템은 물론 비고시 행정관들이 관리하고 있던 자료에서도 모두 대화록이 발견되지 않아 원본은 어디에도 존재하지 않은 것 아니냐는 의혹도 제기했다.
 그러나 정부가 대화록을 원천적으로 국정원으로 넘겼다면 대화록은 국정원의 통제 하에 있었던 상황이다.
 즉 당시 정권은 대통령기록관이 아닌 국정원에서 대화록을 관리해왔다.
 청와대 이지원시스템에 존재하지 않았다는 것 자체가 대화록이 국정원에 있었음을 암시한다.
 국정원도 이를 부인하고 있지 않다.
 국정원은 대화


In [4]:
import os
import torch
import transformers
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
# dataset 가져오기
dataset_koalpaca = load_dataset(kr_dataset)
dataset_koalpaca

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 49620
    })
})

In [6]:
dataset_koalpaca['train'][0]

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
 'input': '',
 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'}

In [7]:
# 데이터프레임으로 변환
df_koalpaca = pd.DataFrame(dataset_koalpaca['train'])

# 중복 제거
df_koalpaca = df_koalpaca.drop_duplicates(keep='first', ignore_index=True)

# HuggingFace Dataset 형태로 변환
dataset_koalpaca = Dataset.from_pandas(df_koalpaca)

print(dataset_koalpaca[0])

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.', 'input': '', 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'}


## Instruction prompt template

In [8]:
prompt_input_template = """
### Instruction : {instruction}

### Input: {input}

### output: {output}"""

In [9]:
def generate_prompt(data_point):
  instruction = data_point["instruction"]
  input = data_point["input"]
  output = data_point["output"]

  res = prompt_input_template.format(instruction=instruction, input=input, output=output)

  data_point['text'] = res

  return data_point

# 데이터셋에 프롬프트 적용
remove_column_keys = dataset_koalpaca.features.keys() # 기존 컬럼(instruction, output 등) 제거
dataset_cvted = dataset_koalpaca.select(range(1)).map(generate_prompt, remove_columns=remove_column_keys)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [10]:
for i in range(1):
    if "건강" in dataset_cvted[i]["text"]:
        print(dataset_cvted[i]["text"])

dataset_cvted


### Instruction : 건강을 유지하기 위한 세 가지 팁을 알려주세요.

### Input: 

### output: 세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.


Dataset({
    features: ['text'],
    num_rows: 1
})

In [11]:
# 양자화 설정 (메모리 절약가능, 빠른 계산이 가능함)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# LLAMA3 basemodel의 model, tokenizer 가져오기
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# 텍스트의 끝을 알려주기 위해 데이터에 EOS 토큰을 추가함
EOS_TOKEN = tokenizer.eos_token
def tokenize_function(examples):
  examples["text"] = "<|begin_of_text|>" + examples["text"] + EOS_TOKEN
  return examples

dataset_tokenized = dataset_cvted.map(tokenize_function)

print(dataset_tokenized[0])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

{'text': '<|begin_of_text|>\n### Instruction : 건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n### Input: \n\n### output: 세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.<|end_of_text|>'}


In [13]:
# LORA configuration
lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# GPU 사용에서 cache를 사용하지 않게 함으로써 메모리 절약
model.config.use_cache = False
model.config.pretraining_tp = 1

In [14]:
train_args = transformers.TrainingArguments(
    output_dir="./results", # output
    num_train_epochs=10, # epoch
    per_device_train_batch_size=4, # batch
    gradient_accumulation_steps=1, # acculmulator
    optim="paged_adamw_32bit", # optimizer
    logging_steps=1, # log 출력
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False, 
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1, # max_step 설정 X (끝날떄 까지)
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_tokenized,
    args=train_args,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    packing=False,
)

c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\transformers\training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\trl\trainer\sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\trl\trainer\sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed w

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [15]:
trainer.train()

  0%|          | 0/10 [00:00<?, ?it/s]

c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 2.6807, 'grad_norm': 0.7231382727622986, 'learning_rate': 0.0002, 'epoch': 1.0}
{'loss': 2.5663, 'grad_norm': 0.6585438251495361, 'learning_rate': 0.0002, 'epoch': 2.0}
{'loss': 2.4237, 'grad_norm': 0.6307597160339355, 'learning_rate': 0.0002, 'epoch': 3.0}
{'loss': 2.2744, 'grad_norm': 0.5859875679016113, 'learning_rate': 0.0002, 'epoch': 4.0}
{'loss': 2.1344, 'grad_norm': 0.6527044773101807, 'learning_rate': 0.0002, 'epoch': 5.0}
{'loss': 1.9897, 'grad_norm': 0.6430408954620361, 'learning_rate': 0.0002, 'epoch': 6.0}
{'loss': 1.8306, 'grad_norm': 0.8425390124320984, 'learning_rate': 0.0002, 'epoch': 7.0}
{'loss': 1.6514, 'grad_norm': 0.9498087167739868, 'learning_rate': 0.0002, 'epoch': 8.0}
{'loss': 1.446, 'grad_norm': 1.052816390991211, 'learning_rate': 0.0002, 'epoch': 9.0}
{'loss': 1.2292, 'grad_norm': 1.1749199628829956, 'learning_rate': 0.0002, 'epoch': 10.0}
{'train_runtime': 13.9731, 'train_samples_per_second': 0.716, 'train_steps_per_second': 0.716, 'train_loss': 2.

TrainOutput(global_step=10, training_loss=2.022618997097015, metrics={'train_runtime': 13.9731, 'train_samples_per_second': 0.716, 'train_steps_per_second': 0.716, 'total_flos': 26212013015040.0, 'train_loss': 2.022618997097015, 'epoch': 10.0})

In [30]:
def generate_response(prompt, model):
    messages = [
        {"role": "system", "content": "친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
        {"role": "user", "content": f"{prompt}"},
    ]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)


instruction = "피타고라스의 정리에 대해 알려줘"

generate_response(instruction, model)

'피타고라스의 정리는 두 직각 삼각형의 빗변의 제곱은 그들의 대응하는 변의 제곱의 합과 같다.assistant\n\n피타고라스의 정리는 두 직각 삼각형의 빗변의 제곱은 그들의 대응하는 변의 제곱의 합과 같다.assistant'

In [17]:
FINETUNED_MODEL = "llama3_LoRA_finetuning_test"
trainer.model.save_pretrained(FINETUNED_MODEL)

c:\Users\Gamzadole\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
